TABLEAU_PROJECT

In [1]:
import pandas as pd 
import pymongo
MongoClient = pymongo.MongoClient

import time

client = MongoClient()
db = client.companies

In [2]:
client.companies

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'companies')

In [3]:
cursor = db.companies.find({"$and": [ {"category_code":"software"} , {"founded_year" : {"$gte": 2000} }, {"offices.latitude" : {"$ne": None} }, {"offices.longitude" : {"$ne": None} } ] },{"name": 1, "company_code": 1,"offices.latitude":1, "offices.longitude":1})
ls_cursor= list(cursor)

#print(ls_cursor)

a = []
counter = 0

for i in ls_cursor:
    i = counter
    a.append(ls_cursor[i])
    counter += 1

In [4]:
df = pd.DataFrame(a)
df.head()

,_id,name,offices
0,52cdef7c4bab8bd675297d8c,Zoho,"[{'latitude': 37.692934, 'longitude': -121.904..."
1,52cdef7c4bab8bd675297e30,GoingOn,"[{'latitude': 37.782263, 'longitude': -122.392..."
2,52cdef7c4bab8bd675297e3e,spigit,"[{'latitude': 37.6637279, 'longitude': -121.87..."
3,52cdef7c4bab8bd675297e3c,Bazaarvoice,"[{'latitude': 30.407545, 'longitude': -97.7176..."
4,52cdef7c4bab8bd675297e57,GoLightly,"[{'latitude': 37.898575, 'longitude': -122.537..."


In [5]:
b = []
counter = 0

for i in df["offices"]:
    i = counter
    b.append(df["offices"][i])
    counter += 1

df_b = pd.DataFrame(b)
df_b = df_b[0] # nos quedamos sólo con la primera sucursal
df_b = pd.Series.to_frame(df_b) # convertimos a DF
df_b.head()

,0
0,"{'latitude': 37.692934, 'longitude': -121.904945}"
1,"{'latitude': 37.782263, 'longitude': -122.392142}"
2,"{'latitude': 37.6637279, 'longitude': -121.873..."
3,"{'latitude': 30.407545, 'longitude': -97.717667}"
4,"{'latitude': 37.898575, 'longitude': -122.537302}"


In [6]:
df_c = pd.concat([df['name'],df_b[0], df['_id']],axis=1)
df_c.head()

,name,0,_id
0,Zoho,"{'latitude': 37.692934, 'longitude': -121.904945}",52cdef7c4bab8bd675297d8c
1,GoingOn,"{'latitude': 37.782263, 'longitude': -122.392142}",52cdef7c4bab8bd675297e30
2,spigit,"{'latitude': 37.6637279, 'longitude': -121.873...",52cdef7c4bab8bd675297e3e
3,Bazaarvoice,"{'latitude': 30.407545, 'longitude': -97.717667}",52cdef7c4bab8bd675297e3c
4,GoLightly,"{'latitude': 37.898575, 'longitude': -122.537302}",52cdef7c4bab8bd675297e57


In [7]:
df_c = df_c.dropna() # eliminamos valores nulos
df_c.head()

,name,0,_id
0,Zoho,"{'latitude': 37.692934, 'longitude': -121.904945}",52cdef7c4bab8bd675297d8c
1,GoingOn,"{'latitude': 37.782263, 'longitude': -122.392142}",52cdef7c4bab8bd675297e30
2,spigit,"{'latitude': 37.6637279, 'longitude': -121.873...",52cdef7c4bab8bd675297e3e
3,Bazaarvoice,"{'latitude': 30.407545, 'longitude': -97.717667}",52cdef7c4bab8bd675297e3c
4,GoLightly,"{'latitude': 37.898575, 'longitude': -122.537302}",52cdef7c4bab8bd675297e57


In [8]:
df_c = df_c.reset_index(drop=False)
df_c.head()

,index,name,0,_id
0,0,Zoho,"{'latitude': 37.692934, 'longitude': -121.904945}",52cdef7c4bab8bd675297d8c
1,1,GoingOn,"{'latitude': 37.782263, 'longitude': -122.392142}",52cdef7c4bab8bd675297e30
2,2,spigit,"{'latitude': 37.6637279, 'longitude': -121.873...",52cdef7c4bab8bd675297e3e
3,3,Bazaarvoice,"{'latitude': 30.407545, 'longitude': -97.717667}",52cdef7c4bab8bd675297e3c
4,4,GoLightly,"{'latitude': 37.898575, 'longitude': -122.537302}",52cdef7c4bab8bd675297e57


In [9]:
def toGeoJSON(array):
    return {
        "type":"Point",
        "coordinates":[array['longitude'],array['latitude']]
    }


df_c[0] = df_c.apply(lambda r: toGeoJSON(r[0]), axis=1)


In [10]:
df_c = df_c.drop(columns=['index','_id'])
df_c.head()

,name,0
0,Zoho,"{'type': 'Point', 'coordinates': [-121.904945,..."
1,GoingOn,"{'type': 'Point', 'coordinates': [-122.392142,..."
2,spigit,"{'type': 'Point', 'coordinates': [-121.8731805..."
3,Bazaarvoice,"{'type': 'Point', 'coordinates': [-97.717667, ..."
4,GoLightly,"{'type': 'Point', 'coordinates': [-122.537302,..."


In [11]:
df_c.to_json('visualize_companies.json', orient="records",lines=True)

In [12]:
cursor = db.df_c.find({
  "position": {
    "$near": {
      "$geometry": {
        "type": "Point",
        "coordinates": [-121.904945, 37.692934]
      }
    }
  }
})
      
list(cursor)

[]